In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('census_data.csv')
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df.income_bracket.value_counts()

 <=50K    24720
 >50K      7841
Name: income_bracket, dtype: int64

In [4]:
def fix_label(label):
    if label == ' <=50K':
        return 0
    else:
        return 1


In [5]:
df['income_bracket'] = df['income_bracket'].apply(fix_label)

In [6]:
df['income_bracket'].unique()

array([0, 1], dtype=int64)

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop('income_bracket', axis = 1)
Y = df['income_bracket']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
import tensorflow as tf

In [10]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [11]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=10)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=100)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=100)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=100)
gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size=100)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size=100)


In [12]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

In [13]:
feature_columns = [age, workclass, education, education_num, marital_status, occupation, relationship, race, gender, capital_gain, capital_loss, hours_per_week, native_country]

In [14]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train,batch_size=128, num_epochs=10, shuffle=False)

In [15]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=2, model_dir='./output')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000002881A57C50>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
model.train(input_fn=input_func)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-4070
INFO:tensorflow:Saving checkpoints for 4071 into ./output\model.ckpt.
INFO:tensorflow:loss = 77.7197, step = 4071
INFO:tensorflow:global_step/sec: 60.4477
INFO:tensorflow:loss = 127.371, step = 4171 (1.661 sec)
INFO:tensorflow:global_step/sec: 113.909
INFO:tensorflow:loss = 42.2807, step = 4271 (0.881 sec)
INFO:tensorflow:global_step/sec: 109.051
INFO:tensorflow:loss = 52.1594, step = 4371 (0.917 sec)
INFO:tensorflow:global_step/sec: 105.895
INFO:tensorflow:loss = 360.235, step = 4471 (0.944 sec)
INFO:tensorflow:global_step/sec: 99.5149
INFO:tensorflow:loss = 56.7626, step = 4571 (1.002 sec)
INFO:tensorflow:global_step/sec: 104.065
INFO:tensorflow:loss = 73.3053, step = 4671 (0.966 sec)
INFO:tensorflow:global_step/sec: 105.786
INFO:tensorflow:loss = 65.3859, step = 4771 (0.940 sec)
INFO:tensorflow:global_step/sec: 106.999
INFO:tensorflow:loss = 49.2812, step = 4871 (0.935 sec)

In [17]:
eval_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, y=y_test, batch_size=128, shuffle=False)
result = model.evaluate(input_fn=eval_fn)

INFO:tensorflow:Starting evaluation at 2018-01-16-17:13:11
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-6105
INFO:tensorflow:Finished evaluation at 2018-01-16-17:13:14
INFO:tensorflow:Saving dict for global step 6105: accuracy = 0.836174, accuracy_baseline = 0.75879, auc = 0.888206, auc_precision_recall = 0.705163, average_loss = 0.439364, global_step = 6105, label/mean = 0.24121, loss = 56.1093, prediction/mean = 0.231479


In [19]:
for key in sorted(result):
  print('%s: %s' % (key, result[key]))

accuracy: 0.836174
accuracy_baseline: 0.75879
auc: 0.888206
auc_precision_recall: 0.705163
average_loss: 0.439364
global_step: 6105
label/mean: 0.24121
loss: 56.1093
prediction/mean: 0.231479


In [20]:
predict_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size=128, shuffle=False)

In [21]:
predict = model.predict(input_fn=predict_fn)

In [22]:
predictions = list(predict)

INFO:tensorflow:Restoring parameters from ./output\model.ckpt-6105


In [24]:
final_prediction = [pred['class_ids'][0] for pred in predictions]

In [25]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(y_test, final_prediction))

             precision    recall  f1-score   support

          0       0.87      0.92      0.90      4942
          1       0.70      0.57      0.63      1571

avg / total       0.83      0.84      0.83      6513

